In [49]:
!pip install streamlit
!pip install pyngrok

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load your dataset
online_sales = pd.read_csv("Online_Sales.csv")
custmer = pd.read_excel("CustomersData.xlsx")

# Merge datasets (adjust as necessary)
data = pd.merge(online_sales, custmer, on="CustomerID", how="left")
data.Transaction_Date = pd.to_datetime(data.Transaction_Date, format = "%Y%m%d")
data.sort_values(by=['CustomerID', 'Transaction_Date'], inplace=True)
# Example logic to create 'Next_Purchase' column
# Assuming 'Purchase_Date' is a column in your online_sales dataset
data['Purchase_Date'] = pd.to_datetime(data['Transaction_Date'])  # Ensure it's in datetime format
data.sort_values(by=['CustomerID', 'Purchase_Date'], inplace=True)

# Create a 'Next_Purchase' column
data['Next_Purchase'] = data.groupby('CustomerID')['Purchase_Date'].shift(-1).notnull().astype(int)

# Prepare your features and target variable
X = data[['Quantity', 'Avg_Price', 'Delivery_Charges']]  # Add other relevant features
y = data['Next_Purchase']  # This is your target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Model saved as model.pkl


In [51]:
# Create a Streamlit app
with open('app.py', 'w') as f:
    f.write(
        """
import streamlit as st
import pandas as pd
import pickle

# Load your pre-trained models
try:
    with open('model.pkl', 'rb') as model_file:
        purchase_model = pickle.load(model_file)
    with open('product_model.pkl', 'rb') as product_model_file:  # Load the new product prediction model
        product_model = pickle.load(product_model_file)
    st.write("Models loaded successfully!")
except Exception as e:
    purchase_model = None
    product_model = None
    st.error(f"Error loading models: {e}")

# Ensure models are loaded before using them
if purchase_model is None or product_model is None:
    st.stop()  # Stop the app if the models are not loaded

# Load data
dis = pd.read_csv("Discount_Coupon.csv")
markt = pd.read_csv("Marketing_Spend.csv")
online_sales = pd.read_csv("Online_Sales.csv")
custmer = pd.read_excel("CustomersData.xlsx")
tax = pd.read_excel("Tax_amount.xlsx")

# Merge datasets
online_sales_custmer = pd.merge(online_sales, custmer, on="CustomerID", how="left")
o_c_t = pd.merge(online_sales_custmer, tax, on="Product_Category", how="left")
o_c_t.Transaction_Date = pd.to_datetime(o_c_t.Transaction_Date, format="%Y%m%d")

# Calculate the date difference
o_c_t = o_c_t.sort_values(by=['CustomerID', 'Transaction_Date'])  # Sort by CustomerID and Transaction_Date
o_c_t['Days_Since_Last_Transaction'] = o_c_t.groupby('CustomerID')['Transaction_Date'].diff().dt.days

# Title
st.title("Customer Segmentation and Next Purchase Prediction")

# Sidebar for user input
st.sidebar.header("User Input Features")

# User input for customer features
customer_id_input = st.sidebar.text_input("Enter Customer ID (or select from the dropdown)")

if customer_id_input:
    try:
        customer_id = int(customer_id_input)
    except ValueError:
        st.error("Cannot make predictions as no data is available for the entered Customer ID")
        st.stop()
else:
    customer_id = st.sidebar.selectbox("Select Customer ID", o_c_t['CustomerID'].unique())

customer_data = o_c_t[o_c_t['CustomerID'] == customer_id]

# Display customer data
if not customer_data.empty:
    st.write("Customer Data:")
    st.write(customer_data)
else:
    st.warning("No data found for the entered Customer ID.")

# Features for prediction
# Assuming these are the features used to train the product_model
expected_product_features = ['Quantity', 'Avg_Price', 'Delivery_Charges']  # Adjust this list based on your model

# Filter X_input to only include the expected features
X_input = customer_data[expected_product_features]

# Button to perform purchase prediction
if st.sidebar.button("Predict Next Purchase"):
    if purchase_model is not None:
        if not customer_data.empty:
            # Predict the next purchase using the pre-trained model
            next_purchase_prediction = purchase_model.predict(X_input)

            # Check the sum of 'Days_Since_Last_Transaction'
            total_days_since_last_transaction = customer_data['Days_Since_Last_Transaction'].sum()

            # Logic to determine likelihood based on days since last transaction
            if total_days_since_last_transaction > 1:
                st.write("Prediction: The customer is likely to make a next purchase (based on transaction history).")
            else:
                # Display the prediction from the model
                if next_purchase_prediction[0] == 1:
                    st.write("Prediction: The customer is likely to make a next purchase (based on model prediction).")
                else:
                    st.write("Prediction: The customer is unlikely to make a next purchase.")
        else:
            st.warning("Cannot make predictions as no data is available for the entered Customer ID.")
    else:
        st.error("Purchase model is not loaded, unable to make predictions.")

# Button to predict next product
if st.sidebar.button("Predict Next Product"):
    if product_model is not None:
        if not customer_data.empty:
            # Predict the next product using the pre-trained product model
            next_product_prediction = product_model.predict(X_input)

            # Display the prediction
            st.write(f"Prediction: The customer is likely to buy {next_product_prediction[0]}.")
        else:
            st.warning("Cannot make predictions as no data is available for the entered Customer ID.")
    else:
        st.error("Product model is not loaded, unable to make predictions.")

"""
    )

In [52]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import pickle

# Assuming 'data' is your dataframe
# Create the 'Next_Product' column, which shifts the 'Product_Category' to get the next purchase
data['Next_Product'] = data.groupby('CustomerID')['Product_Category'].shift(-1)

# Handle missing values by removing rows with NaN in any of the important columns
data = data.dropna(subset=['Quantity', 'Avg_Price', 'Delivery_Charges', 'Next_Product'])

# Prepare features and target variable
X_product = data[['Quantity', 'Avg_Price', 'Delivery_Charges']]  # Add other relevant features
y_product = data['Next_Product']  # This is your target variable

# Train the RandomForest model
product_model = RandomForestClassifier()
product_model.fit(X_product, y_product)

# Save the trained model to a file
with open('product_model.pkl', 'wb') as product_model_file:
    pickle.dump(product_model, product_model_file)

print("Model trained and saved successfully!")


Model trained and saved successfully!


In [53]:
import streamlit as st
import pandas as pd
import pickle

# Load your pre-trained models
try:
    with open('model.pkl', 'rb') as model_file:
        purchase_model = pickle.load(model_file)
    with open('product_model.pkl', 'rb') as product_model_file:  # Load the new product prediction model
        product_model = pickle.load(product_model_file)
    st.write("Models loaded successfully!")
except Exception as e:
    purchase_model = None
    product_model = None
    st.error(f"Error loading models: {e}")

# Ensure models are loaded before using them
if purchase_model is None or product_model is None:
    st.stop()  # Stop the app if the models are not loaded

# Load data
dis = pd.read_csv("Discount_Coupon.csv")
markt = pd.read_csv("Marketing_Spend.csv")
online_sales = pd.read_csv("Online_Sales.csv")
custmer = pd.read_excel("CustomersData.xlsx")
tax = pd.read_excel("Tax_amount.xlsx")

# Merge datasets
online_sales_custmer = pd.merge(online_sales, custmer, on="CustomerID", how="left")
o_c_t = pd.merge(online_sales_custmer, tax, on="Product_Category", how="left")
o_c_t.Transaction_Date = pd.to_datetime(o_c_t.Transaction_Date, format="%Y%m%d")

# Calculate the date difference
o_c_t = o_c_t.sort_values(by=['CustomerID', 'Transaction_Date'])  # Sort by CustomerID and Transaction_Date
o_c_t['Days_Since_Last_Transaction'] = o_c_t.groupby('CustomerID')['Transaction_Date'].diff().dt.days

# Title
st.title("Customer Segmentation and Next Purchase Prediction")

# Sidebar for user input
st.sidebar.header("User Input Features")

# User input for customer features
customer_id = st.sidebar.selectbox("Select Customer ID", o_c_t['CustomerID'].unique())
customer_data = o_c_t[o_c_t['CustomerID'] == customer_id]

# Display customer data
st.write("Customer Data:")
st.write(customer_data)

# Features for prediction
# Assuming these are the features used to train the product_model
expected_product_features = ['Quantity', 'Avg_Price', 'Delivery_Charges']  # Adjust this list based on your model

# Filter X_input to only include the expected features
X_input = customer_data[expected_product_features]

# Button to perform purchase prediction
if st.sidebar.button("Predict Next Purchase"):
    if purchase_model is not None:
        # Predict the next purchase using the pre-trained model
        next_purchase_prediction = purchase_model.predict(X_input)

        # Check the sum of 'Days_Since_Last_Transaction'
        total_days_since_last_transaction = customer_data['Days_Since_Last_Transaction'].sum()

        # Logic to determine likelihood based on days since last transaction
        if total_days_since_last_transaction > 1:
            st.write("Prediction: The customer is likely to make a next purchase (based on transaction history).")
        else:
            # Display the prediction from the model
            if next_purchase_prediction[0] == 1:
                st.write("Prediction: The customer is likely to make a next purchase (based on model prediction).")
            else:
                st.write("Prediction: The customer is unlikely to make a next purchase.")
    else:
        st.error("Purchase model is not loaded, unable to make predictions.")

# Button to predict next product
if st.sidebar.button("Predict Next Product"):
    if product_model is not None:
        # Predict the next product using the pre-trained product model
        next_product_prediction = product_model.predict(X_input)

        # Display the prediction
        st.write(f"Prediction: The customer is likely to buy {next_product_prediction[0]}.")
    else:
        st.error("Product model is not loaded, unable to make predictions.")


2025-02-09 07:55:30.875 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:30.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:30.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:30.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:31.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:31.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:31.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 07:55:31.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [54]:
from pyngrok import ngrok

# Set up ngrok authentication (replace with your actual token)
!ngrok authtoken "2sdUCVAqBtQ9Q3z4nzibCdbMqDi_3zFPL2qMszQHDrsoQhnFe"  # Replace with your actual token

# Set up a tunnel to the streamlit app
public_url = ngrok.connect("http://localhost:8501")
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:8501\"")

# Run the Streamlit app
!streamlit run app.py &


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * ngrok tunnel "NgrokTunnel: "https://e534-34-48-129-129.ngrok-free.app" -> "http://localhost:8501"" -> "http://127.0.0.1:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.48.129.129:8501

  Stopping...
